In [1]:
#imports
import spacy
import random
import numpy as np
import pickle
import pandas as pd
import itertools
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise
from sklearn.cluster import KMeans
from collections import Counter

In [2]:
with open("wordlist.pkl", "rb") as f:
    words = pickle.load(f)

In [3]:
with open("word_lexicals.pkl", "rb") as f:
    texts = pickle.load(f)

In [4]:
df = pd.read_excel("Lexique383.xlsb", engine='pyxlsb')

In [5]:
nlp = spacy.load("fr_core_news_lg")

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise

In [7]:
sample_words = random.sample(words, 2)

In [8]:
def calculate_distance(a, b, option="cosine"):
    if option == "euclidean":
        return np.linalg.norm(a-b)
    if option == "scalar":
        return a @ b
    if option == "cosine":
        return pairwise.cosine_similarity(a.reshape(1,-1), b.reshape(1,-1))[0][0]

In [9]:
def calculate_proximity_score(word_candidate, word_board):
    w1 = nlp.vocab.get_vector(str(word_candidate))
    w2 = nlp.vocab.get_vector(str(word_board))
    if word_candidate in texts[word_board]:
        return 1 + calculate_distance(w1,w2)
    return calculate_distance(w1, w2)

In [10]:
def get_score_df(pos, neg, neu, assassin, indices, danger_coeff):
    
    all_words_candidate = []
    for w in pos:
        for lexical_w in texts[w]:
            #if lexical_word is too close to pos word, do not add it
            if lexical_w[:min(4, len(w))] == w[:min(4, len(w))]:
                continue
            all_words_candidate.append(lexical_w)
            
    pos_dist = [[calculate_proximity_score(word_candidate, word_board)for word_board in pos] for word_candidate in all_words_candidate]
    neg_dist = [[calculate_proximity_score(word_candidate, word_board)for word_board in neg] for word_candidate in all_words_candidate]
    neu_dist = [[calculate_proximity_score(word_candidate, word_board)for word_board in neu] for word_candidate in all_words_candidate]
    ass_dist = [calculate_proximity_score(word_candidate, assassin) for word_candidate in all_words_candidate]
    
    
    #build result df, containing candidate and their scores (positive and negative)
    df = pd.DataFrame(pos_dist, index=all_words_candidate)
    #compute a score for each row (a candidate)
    #df["score"] = df.apply(lambda x: np.product(x) / np.sum(x), axis=1)
    
    df["neg"] = np.max(neg_dist, axis=1) * danger_coeff
    if len(neu) > 0:
        #otherwise, no neutrals
        df["neu"] = np.max(neu_dist, axis=1) 
    df["ass"] = np.array(ass_dist) * danger_coeff
    
    #remove words
    for w in np.concatenate([pos, indices]):
        if w.lower() in all_words_candidate:
            df.drop(w, inplace=True)
    
    df.drop_duplicates(inplace=True)
    
    return df

In [16]:
def calculate_clue_score(clue, pos, neu, agg, df, min_words_to_find):
    scores = df.loc[df.index == clue]
    
    if len(neu) == 0:
        if scores[["neg", "ass"]].max(axis=1)[0] >= 1:
            return -1, None
    else:
        if scores[["neg", "neu", "ass"]].max(axis=1)[0] >= 1:
            return -1, None
        
    # Order scores by highest to lowest (highest is best
    pos_scores = scores.iloc[:, :len(pos)].values[0]
    boundary = np.max(scores.iloc[:,len(pos):], axis=1).values[0]
    i = 0
    
    min_size_group = 0
    if min_words_to_find:
        #let's go all in
        min_size_group = len(pos)
    
    # Order scores by lowest to highest inner product with the clue.
    ss = sorted((s, i) for i, s in enumerate(pos_scores))
    groups = []
    groups_score = []
    for i in range(min_size_group, len(pos_scores)+1):
        group = ss[-(i+1):]
        group_score = []
        for tpl in group:
            group_score.append(tpl[0])

            # Calculate the "real score" by
            #    (lowest score in group) * [ (group size)^aggressiveness - 1].
            # The reason we subtract one is that we never want to have a group of
            # size 1.
        groups_score.append((np.min(group_score) - (boundary/len(group))) * (len(group)**agg - 0.99))
        groups.append([tpl[1] for tpl in group])
    
    real_score = max(groups_score)
    word_indices = groups[np.argmax(groups_score)]  
    
    
    return real_score, word_indices

In [17]:
def get_clue(pos_words, neg_words, neu_words, assassin_word, danger_coeff=1.2, agg=0.02, given_indices=[]):
    
    best_clue, best_score, words_to_guess = None, -1 , None
    
    min_words_to_find = False
    if len(neg_words) < 3:
        print("ALL IN")
        #then opponents is likely to win next round. We should go all in
        min_words_to_find = True
    
    df = get_score_df(pos_words, neg_words, neu_words, assassin_word, given_indices, danger_coeff)
    
    for candidate in df.index:
        real_score, word_ind = calculate_clue_score(candidate, pos_words, neu_words, agg, df, min_words_to_find)
        if real_score > best_score:
            best_clue = candidate
            best_score = real_score
            words_to_guess = np.array(pos_words)[word_ind]
            #print(f"clue: {best_clue}, words: {words_to_guess}, score:{best_score}")
    

    return best_clue, best_score, words_to_guess


In [18]:
#Build plateau
class Plateau():

    #constructor
    def __init__(self, B, R, N, A, status):

        #list of words
        self.B = list(B)
        self.R = list(R)
        self.N = list(N)
        self.A = A

        #Status = 1. If 0, then game is over
        self.status = status

        #all words
        lst = np.concatenate([self.B, self.R, self.N], axis=0)
        words = np.concatenate([lst, [self.A]])
        random.shuffle(words)
        self.words = words
        


    #methods
    def update_status(self):
        """
        This method uses the inputs (words and their associated team to build new attributes: list of words for each team.
        This should be used after every update of the Plateau
        """

        #If one of these is empty: game is over. Status becomes 0, it ends the game
        if min([len(self.B), len(self.R), len(self.A)]) == 0:
            self.status = 0
            print("Game is over")

        #print(f"There is {len(self.B)} words for team BLUE\n")
        #print(f"There is {len(self.R)} words for team RED\n")
        #print(f"There is {len(self.N)} neutral words\n")
        #print(f"There is {len(self.A)} assasin\n")

        return None

    def remove_word(self, chosen_word):
        """
        This function takes a word that the user wants to remove. It prompts a message and then proceeds to remove it.
        """
        #check if in list
        if chosen_word == "next":
            print("\nYou decided to pass.\n")
            return None
        
        if chosen_word not in self.words:
            return self.remove_word(chosen_word=input("Choose another one, this one is not in the board\n>>"))
        
        else:
            self.words = np.where(self.words==chosen_word, "X", self.words) 
        #print(f" \nYou decided to remove {chosen_word}\n")

        #remove
        if chosen_word in self.B:
            print('It was a BLUE word!\n')
            self.B.remove(chosen_word)

        if chosen_word in self.R:
            print('It was a RED word!\n')
            self.R.remove(chosen_word)           

        if chosen_word in self.N:
            print('It was a NEUTRAL word!\n')
            self.N.remove(chosen_word)

        if chosen_word in self.A:
            print('It was the ASSASSIN word!\n')
            self.A = []

        return None


In [19]:
def generate_board():
    
    rs = np.array(random.sample(words, 25))
    print("BOARD\n")
    print(rs.reshape(5,5))
    
    np.random.shuffle(rs)
  
    B = rs[:8]
    R = rs[8:17]
    N = rs[17:24]
    A = rs[24]
    
    return B, R, N, A

In [20]:
#Lauch game!
B, R, N, A = generate_board()
plateau = Plateau(B, R, N, A, 1)
i = 0

given_indices = []

INDICES = {}

NB_TOURS = 0
while plateau.status == 1:
    i = (-1)*i + 1
    team_name = ["BLUE", "RED"][i]
    NB_TOURS += 1
    plateau.update_status()
    team = [plateau.B, plateau.R][i]
    opponent = [plateau.R, plateau.B][i]
    print("__________________")
    print(f"This turn, you are {team_name}")
    print(f"Words remaining for team: {len(team)}")
    print(f"Words remaining for opponent: {len(opponent)}\n")
    
    if i == 1:
        clue, score, group = get_clue(team, opponent, plateau.N, plateau.A, given_indices=given_indices)
    if i == 0:
        clue, score, group = get_clue(team, opponent, plateau.N, plateau.A, given_indices=given_indices)
    
    INDICES[clue] = list(group)
    given_indices.append(clue)
    print(f"The given clue: {clue}")
    print(f"Words to find: {len(group)}\n")
    for k in range(len(group)):
        print(plateau.words.reshape(5,5))
        w = input("Please chose a word to remove\n>>")
        if w in opponent or w in plateau.N:
            plateau.remove_word(w)
            print("End of turn")
            break
        
        plateau.remove_word(w)
        plateau.update_status()
        if plateau.status == 0:
            break
            
    plateau.update_status()
    
print("____________________\n")
print("____________________\n")
print("____________________\n")
print(f"NB TOURS: {NB_TOURS}")

for c in INDICES:
    print(f"Clue: {c}, words to get: {INDICES[c]}")

BOARD

[['surface' 'gorge' 'lentille' 'cochon' 'top']
 ['croûte' 'long' 'jambon' 'fou' 'café']
 ['ligne' 'soldat' 'botte' 'planète' 'chausson']
 ['saison' 'nuit' 'boule' 'franc' 'satellite']
 ['kilo' 'camping' 'thé' 'lecteur' 'mystère']]
__________________
This turn, you are RED
Words remaining for team: 9
Words remaining for opponent: 8

The given clue: grillé
Words to find: 3

[['top' 'kilo' 'botte' 'chausson' 'café']
 ['jambon' 'nuit' 'gorge' 'lecteur' 'satellite']
 ['thé' 'croûte' 'long' 'fou' 'soldat']
 ['franc' 'camping' 'surface' 'saison' 'boule']
 ['mystère' 'planète' 'ligne' 'cochon' 'lentille']]
Please chose a word to remove
>>jambon
It was a RED word!

[['top' 'kilo' 'botte' 'chausson' 'café']
 ['X' 'nuit' 'gorge' 'lecteur' 'satellite']
 ['thé' 'croûte' 'long' 'fou' 'soldat']
 ['franc' 'camping' 'surface' 'saison' 'boule']
 ['mystère' 'planète' 'ligne' 'cochon' 'lentille']]
Please chose a word to remove
>>croûte
It was a NEUTRAL word!

End of turn
__________________
This tur

## Evaluation AI NLP

In [103]:
def generate_board_var(board_size):
    
    if board_size == 25:
        rs = np.array(random.sample(words, 25))    
        np.random.shuffle(rs)  
        J1_words = rs[:8]
        J2_words = rs[8:17]
        N_words = rs[17:24]
        A_words = rs[24]
    
    elif board_size == 20:
        rs = np.array(random.sample(words, 20))   
        np.random.shuffle(rs)  
        J1_words = rs[:6]
        J2_words = rs[6:12]
        N_words = rs[12:19]
        A_words = rs[19]
    
    elif board_size == 15:
        rs = np.array(random.sample(words, 15))    
        np.random.shuffle(rs)  
        J1_words = rs[:4]
        J2_words = rs[4:8]
        N_words = rs[8:14]
        A_words = rs[14]
    
    elif board_size == 10:
        rs = np.array(random.sample(words, 10)) 
        np.random.shuffle(rs)  
        J1_words = rs[:3]
        J2_words = rs[3:6]
        N_words = rs[6:9]
        A_words = rs[9]
    
    return J1_words, J2_words, N_words, A_words

In [104]:
def evaluation_nlp(n_iterations=1):
    
    n_J1, n_J2, n_N, n_A = 0, 0, 0, 0
    
    for i in range(n_iterations):
        
        list_sizes = [25, 20, 15, 10]
        for board_size in list_sizes:
            pos_words, neg_words, neu_words, assassin_word = generate_board_var(board_size)
            plateau = Plateau(pos_words, neg_words, neu_words, assassin_word, 1)
            
            clue, score, group = get_clue(pos_words, neg_words, neu_words, assassin_word, given_indices=[])
            
            print(f"The given clue: {clue}")
            print(f"Words to find: {len(group)}\n")
            for k in range(len(group)):
                print(plateau.words.reshape(board_size//5,5))
                w = input("Please chose a word to remove\n>>")
                while w not in plateau.words:
                    w = input("Please chose a word from the board\n>>")
                if w == "next":
                    print("You decided to pass.")
                    break
                elif w in plateau.R:
                    print("Negative word")
                    n_J2 += 1
                    break
                elif w in plateau.N:
                    print("Neutral word")
                    n_N += 1
                    break
                elif w in plateau.A:
                    print("Assassin word")
                    n_A += 1
                    break
                else: 
                    print("Positive word")
                    n_J1 += 1        
                    plateau.remove_word(w)
                    plateau.update_status()
                    if plateau.status == 0:
                        break
    
    n_J1 /= 4*n_iterations
    n_J2 /= 4*n_iterations
    n_N /= 4*n_iterations
    n_A /= 4*n_iterations
            
    return n_J1, n_J2, n_N, n_A        

In [106]:
n_J1, n_J2, n_N, n_A = evaluation_nlp()

The given clue: longueur
Words to find: 3

[['chauve' 'religieuse' 'surface' 'saut' 'monstre']
 ['bière' 'caméra' 'court' 'chance' 'partie']
 ['marque' 'plastique' 'canon' 'reptile' 'outil']
 ['son' 'himalaya' 'astérix' 'titre' 'veste']
 ['plateau' 'police' 'ensemble' 'cassette' 'macho']]
Please chose a word to remove
>>surface
Positive word
It was a BLUE word!

[['chauve' 'religieuse' 'X' 'saut' 'monstre']
 ['bière' 'caméra' 'court' 'chance' 'partie']
 ['marque' 'plastique' 'canon' 'reptile' 'outil']
 ['son' 'himalaya' 'astérix' 'titre' 'veste']
 ['plateau' 'police' 'ensemble' 'cassette' 'macho']]
Please chose a word to remove
>>saut
Positive word
It was a BLUE word!

[['chauve' 'religieuse' 'X' 'X' 'monstre']
 ['bière' 'caméra' 'court' 'chance' 'partie']
 ['marque' 'plastique' 'canon' 'reptile' 'outil']
 ['son' 'himalaya' 'astérix' 'titre' 'veste']
 ['plateau' 'police' 'ensemble' 'cassette' 'macho']]
Please chose a word to remove
>>court
Positive word
It was a BLUE word!

The given c

In [107]:
def agregation_NLP(n_J1, n_J2, n_N, n_A):
    print("IA score: ", n_J1 - n_J2 - 3*n_A)
    return n_J1 - n_J2 - 3*n_A

In [108]:
agregation_NLP(n_J1, n_J2, n_N, n_A)

IA score:  1.25


1.25

## Evaluation AI In-Game

In [113]:
def evaluation_ingame(n_iterations=2):
    
    dict_scores = {'n_J1': [], 'n_J2': [], 'n_A': [], 'T': []}
    
    for i in range(1, n_iterations+1):
        
        print(f"Game {i}/{n_iterations}")
        print("You play the RED team")
        n_J1, n_J2, n_A, T = 0, 0, 0, 0
    
        B, R, N, A = generate_board()
        plateau = Plateau(B, R, N, A, 1)
        turn = 1

        given_indices = []
        
        while plateau.status == 1:

            plateau.update_status()
            team = plateau.R
            opponent = plateau.B
            
            print(f"___Turn {turn}____")
            print(f"Words remaining for team: {len(team)}")
            print(f"Words remaining for opponent: {len(opponent)}\n")
            
            # Turn of J1 (Red)
            clue, score, group = get_clue(team, opponent, plateau.N, plateau.A, given_indices=given_indices)
    
            given_indices.append(clue)
            print(f"The given clue: {clue}")
            print(f"Words to find: {len(group)}\n")
        
            for k in range(len(group)):
            
                print(plateau.words.reshape(5,5))
                w = input("Please chose a word to remove\n>>")
                
                if w in opponent:
                    n_J2 += 1
                    plateau.remove_word(w)
                    print("End of turn")
                    break
                    
                if w in plateau.N:
                    plateau.remove_word(w)
                    print("End of turn")
                    break
        
                if w in plateau.A:
                    n_A += 1
                    plateau.remove_word(w)
                    print("End of turn")
                    break
                    
                if w in plateau.R:
                    n_J1 += 1
                    plateau.remove_word(w)
                    
            plateau.update_status()        
            if plateau.status == 0:
                break
                    
            # Turn of J2 (Blue)
            if turn % 2 == 1 :
                remove_J2 = random.choice(plateau.B)
                print("Word removed for J2: ", remove_J2)
                plateau.remove_word(remove_J2)
                if plateau.status == 0:
                    break
                remove_J2 = random.choice(plateau.B)
                print("Word removed for J2: ", remove_J2)
                plateau.remove_word(remove_J2)
                if plateau.status == 0:
                    break
                if len(plateau.N) > 0:
                    remove_N = random.choice(plateau.N)
                    print("Word removed for N: ", remove_N)
                    plateau.remove_word(remove_N)
                    
            if turn % 2 == 0 :
                remove_J2 = random.choice(plateau.B)
                print("Word removed for J2: ", remove_J2)
                plateau.remove_word(remove_J2)
                if plateau.status == 0:
                    break
                remove_J1 = random.choice(plateau.R)
                print("Word removed for J1: ", remove_J1)
                plateau.remove_word(remove_J1)
                if plateau.status == 0:
                    break                  
                    
            turn += 1     
        
        print(f"Score for this game: n_J1 = {n_J1}, n_J2 = {n_J2}, n_A = {n_A}, T = {turn}")
        
        dict_scores['n_J1'].append(n_J1)
        dict_scores['n_J2'].append(n_J2) 
        dict_scores['n_A'].append(n_A)  
        dict_scores['T'].append(turn)
                    
    return dict_scores

In [114]:
dict_scores = evaluation_ingame(n_iterations=2)

Game 1/2
You play the RED team
BOARD

[['stylo' 'lecteur' 'penser' 'poète' 'vache']
 ['plaie' 'sang' 'portable' 'couvert' 'ciel']
 ['voiture' 'vœu' 'roulette' 'bord' 'vingt']
 ['œuf' 'barre' 'sieste' 'égout' 'tambour']
 ['cuisine' 'charbon' 'traîner' 'nature' 'musique']]
___Turn 1____
Words remaining for team: 9
Words remaining for opponent: 8

The given clue: bille
Words to find: 3

[['cuisine' 'sieste' 'portable' 'plaie' 'tambour']
 ['œuf' 'poète' 'musique' 'barre' 'vache']
 ['bord' 'traîner' 'lecteur' 'stylo' 'voiture']
 ['vœu' 'roulette' 'sang' 'nature' 'égout']
 ['couvert' 'vingt' 'charbon' 'penser' 'ciel']]
Please chose a word to remove
>>roulette
It was a RED word!

[['cuisine' 'sieste' 'portable' 'plaie' 'tambour']
 ['œuf' 'poète' 'musique' 'barre' 'vache']
 ['bord' 'traîner' 'lecteur' 'stylo' 'voiture']
 ['vœu' 'X' 'sang' 'nature' 'égout']
 ['couvert' 'vingt' 'charbon' 'penser' 'ciel']]
Please chose a word to remove
>>stylo
It was a RED word!

[['cuisine' 'sieste' 'portable' '

IndexError: Cannot choose from an empty sequence

In [ ]:
def agregation_ingame(dict_scores):
    score, iters = 0, len(dict_scores['n_J1'])
    for i in range(iters):
        score += ((dict_scores['n_J1'][i] - dict_scores['n_J2'][i] - 2*dict_scores['n_A'][i])/dict_scores['T'][i])
    score /= iters
    print("IA score: ", score)
    return score

In [ ]:
agregation_ingame(dict_scores)